In [1]:
!apt-get install -y python-numpy python-dev cmake zlib1g-dev libjpeg- dev xvfb libav-tools xorg-dev python-opengl libboost-all-dev libsdl2-dev swig

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package libav-tools is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source
However the following packages replace it:
  ffmpeg

E: Unable to locate package libjpeg
E: Unable to locate package dev
E: Package 'libav-tools' has no installation candidate


In [2]:
!apt-get install xvfb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 22 not upgraded.
Need to get 783 kB of archives.
After this operation, 2,266 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.6 [783 kB]
Fetched 783 kB in 2s (359 kB/s)
Selecting previously unselected package xvfb.
(Reading database ... 144618 files and directories currently installed.)
Preparing to unpack .../xvfb_2%3a1.19.6-1ubuntu4.6_amd64.deb ...
Unpacking xvfb (2:1.19.6-1ubuntu4.6) ...
Setting up xvfb (2:1.19.6-1ubuntu4.6) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [3]:
!pip install pyvirtualdisplay
!pip install piglet

     |████████████████████████████████| 71kB 1.9MB/s 
     |████████████████████████████████| 92kB 3.4MB/s 


In [4]:
!pip install gym
!pip install “gym[atari]"

/bin/bash: -c: line 0: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [5]:
import gym
import numpy as np
import tensorflow as tf
import random
from collections import deque

In [6]:
def replay_train(mainDQN, targetDQN, train_batch):
  
  x_stack = np.empty(0).reshape(0,input_size)
  y_stack = np.empty(0).reshape(0,output_size)
  
  for state, action, reward, next_state, done in train_batch:
    Q = mainDQN.predict(state)
    
    if done:
      Q[0,action] = reward
    else:
      Q[0,action] = reward + dis*np.max(targetDQN.predict(next_state))
      
    x_stack = np.vstack([x_stack, state])
    y_stack = np.vstack([y_stack, Q])
  
  return mainDQN.update(x_stack,y_stack)

In [7]:
def get_copy_var_ops(*,dest_scope_name = "target", src_scope_name="main"):
  op_holder = []
  
  src_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = src_scope_name)
  dest_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = dest_scope_name)
  
  for src_var, dest_var in zip(src_vars,dest_vars):
    op_holder.append(dest_var.assign(src_var.value()))
  
  return op_holder

In [8]:
def bot_play(mainDQN):
  s=env.reset()
  reward_sum = 0
  while True:
    env.render()
    a = np.argmax(mainDQN.predict(s))
    s, reward, done = env.step(a)
    reward_sum += reward
    
    if done:
      print("Total score : {}".format(reward_sum))
      break;

In [9]:
class DQN:
  
  def __init__(self, session, input_size, output_size, name="main"):
    self.session = session
    self.input_size = input_size
    self.output_size = output_size
    self.net_name = name
    self.build_network()
  
  def build_network(self, h_size = 10, lr = 1e-1):
    with tf.variable_scope(self.net_name):
      
      self.X = tf.placeholder(tf.float32, [None, input_size], name = "input_x")
      W1 = tf.get_variable("W1", shape=[self.input_size, h_size], initializer=tf.contrib.layers.xavier_initializer())
      layer1 = tf.nn.relu(tf.matmul(self.X,W1))
      W2 = tf.get_variable("W2", shape=[h_size, output_size], initializer=tf.contrib.layers.xavier_initializer())
      
      self.Q_pred = tf.matmul(layer1,W2)
    
    self.Y = tf.placeholder(shape = [None,self.output_size], dtype = tf.float32)
    
    self.cost = tf.reduce_mean(tf.square(self.Y - self.Q_pred))
    
    self.train = tf.train.AdamOptimizer(learning_rate = lr).minimize(self.cost)
  
  def predict(self, state):
    x = np.reshape(state, [1, self.input_size])
    return self.session.run(self.Q_pred, feed_dict = {self.X : x})
  
  def update(self, x_stack, y_stack):
    return self.session.run([self.cost, self.train], feed_dict = {self.X : x_stack, self.Y : y_stack})

In [10]:
tf.reset_default_graph()
env = gym.make('CartPole-v0')

input_size = env.observation_space.shape[0]
output_size = env.action_space.n

dis = 0.9
REPLAY_MEMORY = 50000

max_episode = 5000
replay_buffer = deque()

with tf.Session() as sess:
  mainDQN = DQN(sess, input_size, output_size, name = "main")
  targetDQN = DQN(sess, input_size, output_size, name = "target")
  tf.global_variables_initializer().run()
  
  copy_ops = get_copy_var_ops(dest_scope_name="target",src_scope_name="main")
  sess.run(copy_ops)
  
  for episode in range(max_episode):
    e=1./((episode/10)+1)
    done = False
    step_count = 0
    state = env.reset()
    
    while not done:
      if np.random.rand(1) < e:
        action = env.action_space.sample()
      else:
        action = np.argmax(mainDQN.predict(state))
        
      next_state, reward, done, _ = env.step(action)
      
      if done:
        reward = -100
      
      replay_buffer.append((state, action, reward, next_state, done))
      if len(replay_buffer) > REPLAY_MEMORY:
        replay_buffer.popleft()
        
      state = next_state
      step_count += 1
      
      if step_count > 10000:
        break;
      
    print("Episode : {} step : {}".format(episode, step_count))
    
    if episode%10 == 1:
      for _ in range(50):
        minibatch = random.sample(replay_buffer,10)
        cost,_ = replay_train(mainDQN, targetDQN, minibatch)
      print("cost : ",cost)
      sess.run(copy_ops)
#bot_play(mainDQN)

AttributeError: ignored